In [1]:
%pip install ultralytics
%pip install supervision
import os
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from ultralytics import YOLO
import supervision as sv
from tqdm import tqdm
import torch


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:

# ---------- CONFIG ----------
DATA_ROOT = "TU-DAT"
PROCESSED_ROOT = os.path.join(DATA_ROOT, "Final Videos_processed")
OUTPUT_CSV = "trajectories.csv"  # All tracks across all videos
TRAJS_DIR = "trajectories"       # Per-video JSON tracks

TARGET_FPS = 12  # matches your preprocessing
PIXEL_TO_METER = 0.1  # rough calibration: 1 pixel = 0.1m (adjust later)
# -----------------------------


In [3]:
def ensure_dir(path: str):
    Path(path).mkdir(parents=True, exist_ok=True)

In [ ]:
class TrajectoryExtractor:
    def __init__(self):
        # Load YOLO + ByteTrack
        self.model = YOLO('yolov8n.pt')
        self.tracker = sv.ByteTrack()
        
        # Object classes from COCO (person=0, car=2, motorcycle=3, bus=5, truck=7)
        self.object_classes = [0, 2, 3, 5, 7]  # person, car, motorcycle, bus, truck
        
        # Store all trajectories
        self.all_trajectories = []
        
    def pixel_velocity(self, prev_pos, curr_pos, dt_fps):
        """Velocity in m/s from pixel displacement"""
        dx = curr_pos[0] - prev_pos[0]
        dy = curr_pos[1] - prev_pos[1]
        pixel_speed = np.sqrt(dx**2 + dy**2)
        meter_speed = pixel_speed * PIXEL_TO_METER
        speed_ms = meter_speed * TARGET_FPS  # pixels/sec → m/s
        return speed_ms, np.degrees(np.arctan2(dy, dx))
    
    def process_video(self, video_path):
        """Extract trajectories from one video"""
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"[WARN] Cannot open {video_path}")
            return
        
        video_name = Path(video_path).name
        video_full_name = f"{Path(video_path).parent.name}/{video_name}"
        print(f"\nProcessing {video_full_name}...")
        
        # Setup video writer for tracked video
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        tracked_dir = "tracked_videos"
        ensure_dir(tracked_dir)
        tracked_path = os.path.join(tracked_dir, f"{Path(video_path).parent.name}_{Path(video_name).stem}_tracked.mp4")
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(tracked_path, fourcc, fps, (width, height))
        
        frame_trajectories = []
        frame_idx = 0
        track_history = {}  # track_id: list of (cx, cy)
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
                
            # YOLO detection
            results = self.model(frame, verbose=False, conf=0.3)[0]
            detections = sv.Detections.from_ultralytics(results)
            
            # Filter to objects only
            object_mask = np.isin(detections.class_id, self.object_classes)
            detections = sv.Detections(
                xyxy=detections.xyxy[object_mask],
                confidence=detections.confidence[object_mask],
                class_id=detections.class_id[object_mask],
                tracker_id=detections.tracker_id[object_mask] if detections.tracker_id is not None else None
            )
            
            # ByteTrack
            tracks = self.tracker.update_with_detections(detections)
            
            # Extract track data
            frame_data = []
            for track_id in tracks.tracker_id:
                if track_id is None:
                    continue
                    
                # Bounding box center
                x1, y1, x2, y2 = tracks.xyxy[tracks.tracker_id == track_id][0]
                cx, cy = (x1 + x2) / 2, (y1 + y2) / 2
                
                # Update track history
                if track_id not in track_history:
                    track_history[track_id] = []
                track_history[track_id].append((int(cx), int(cy)))
                
                # Class name
                cls_id = tracks.class_id[tracks.tracker_id == track_id][0]
                cls_name = self.model.names[cls_id]
                
                # Velocity (simple: compare to previous frame)
                speed = 0.0
                heading = 0.0
                if len(frame_trajectories) > 0:
                    prev_frame = frame_trajectories[-1]
                    prev_pos = next(((t['center_x'], t['center_y']) for t in prev_frame if t['track_id'] == track_id), None)
                    if prev_pos:
                        speed, heading = self.pixel_velocity(prev_pos, (cx, cy), 1.0 / TARGET_FPS)
                
                frame_data.append({
                    'frame': frame_idx,
                    'track_id': int(track_id),
                    'class': cls_name,
                    'bbox': [float(x1), float(y1), float(x2), float(y2)],
                    'center_x': float(cx),
                    'center_y': float(cy),
                    'speed_mps': float(speed),
                    'heading_deg': float(heading),
                    'video': video_full_name,
                    'label': Path(video_path).parent.name
                })
            
            frame_trajectories.append(frame_data)
            frame_idx += 1
            
            # Annotate frame with detections and trajectories
            annotated_frame = results.plot()
            
            # Draw trajectories
            for track_id, points in track_history.items():
                if len(points) > 1:
                    pts = np.array(points, np.int32).reshape((-1, 1, 2))
                    cv2.polylines(annotated_frame, [pts], False, (0, 255, 0), 2)
            
            # Show preview (optional)
            cv2.imshow('Tracking', annotated_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        video_writer.release()
        cv2.destroyAllWindows()
        
        # Save per-video trajectories
        video_trajs_path = os.path.join(TRAJS_DIR, f"{Path(video_path).parent.name}_{Path(video_name).stem}_tracks.json")
        ensure_dir(TRAJS_DIR)
        import json
        with open(video_trajs_path, 'w') as f:
            json.dump(frame_trajectories, f)
        
        # Flatten for global CSV
        flat_trajectories = []
        for frame_data in frame_trajectories:
            flat_trajectories.extend(frame_data)
        self.all_trajectories.extend(flat_trajectories)
        
        print(f"  → {len(flat_trajectories)} detections across {frame_idx} frames")
        print(f"  → Saved tracked video to {tracked_path}")
        return frame_trajectories

In [5]:
# This code snippet is now part of the TrajectoryExtractor class above.
# It can be removed.

In [6]:
def main():
    # Install dependencies first
    os.system("pip install ultralytics supervision")
    
    extractor = TrajectoryExtractor()
    
    # Process all processed videos
    positive_dir = os.path.join(PROCESSED_ROOT, "positive")
    negative_dir = os.path.join(PROCESSED_ROOT, "negative")
    
    all_videos = []
    for folder in [positive_dir, negative_dir]:
        if os.path.isdir(folder):
            videos = [os.path.join(folder, f) for f in os.listdir(folder) 
                     if f.endswith('.mp4')]
            all_videos.extend(videos)
    
    print(f"Found {len(all_videos)} videos to process")
    
    # Process all videos
    for video_path in tqdm(all_videos):
        extractor.process_video(video_path)
    
    # Save global trajectories CSV
    df = pd.DataFrame(extractor.all_trajectories)
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"\n✅ Saved {len(df)} trajectories to {OUTPUT_CSV}")
    
    if not df.empty:
        print("\nSample data:")
        print(df[['video', 'label', 'frame', 'track_id', 'class', 'speed_mps', 'heading_deg']].head())
    else:
        print("No trajectories extracted.")

main()

Found 98 videos to process


  0%|          | 0/98 [00:00<?, ?it/s]


Processing v10_proc.mp4...


  1%|          | 1/98 [00:07<12:50,  7.95s/it]

  → 1203 detections across 70 frames

Processing v11_proc.mp4...


...

  → 80 detections across 25 frames

Processing v9_proc.mp4...


100%|██████████| 98/98 [11:07<00:00,  6.81s/it]

  → 94 detections across 38 frames



✅ Saved 25322 trajectories to trajectories.csv

Sample data:


KeyError: "['label'] not in index"

In [ ]:
import os
print("DATA_ROOT:", DATA_ROOT)
print("PROCESSED_ROOT:", PROCESSED_ROOT)
positive_dir = os.path.join(PROCESSED_ROOT, "positive")
negative_dir = os.path.join(PROCESSED_ROOT, "negative")
print("positive_dir:", positive_dir, "exists:", os.path.isdir(positive_dir))
print("negative_dir:", negative_dir, "exists:", os.path.isdir(negative_dir))
if os.path.isdir(positive_dir):
    videos = [f for f in os.listdir(positive_dir) if f.endswith('.mp4')]
    print("Videos in positive:", videos[:5])  # first 5

DATA_ROOT: TU-DAT
PROCESSED_ROOT: TU-DAT\Final Videos_processed
positive_dir: TU-DAT\Final Videos_processed\positive exists: True
negative_dir: TU-DAT\Final Videos_processed\negative exists: True
Videos in positive: ['v10_proc.mp4', 'v11_proc.mp4', 'v12_proc.mp4', 'v13_proc.mp4', 'v14_proc.mp4']
